In [ ]:
# Laboratorio 2: Ataque FGSM básico en MNIST a CNN
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

# Cargar y preparar los datos
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Crear un modelo CNN simple
def crear_modelo():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Entrenar el modelo (o cargar uno pre-entrenado)
model = crear_modelo()
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=1)

# Función para crear ejemplos adversarios usando FGSM
def crear_ejemplo_adversario_fgsm(modelo, imagen, etiqueta, epsilon=0.1):
    # Guardar la forma original para restaurarla al final
    forma_original = imagen.shape

    # Preparar la imagen para el modelo (añadiendo dimensión de batch)
    imagen = tf.convert_to_tensor(imagen.reshape(1, 28, 28, 1), dtype=tf.float32)
    etiqueta = tf.convert_to_tensor([etiqueta], dtype=tf.int64)

    with tf.GradientTape() as tape:
        tape.watch(imagen)
        prediccion = modelo(imagen)
        loss = tf.keras.losses.sparse_categorical_crossentropy(etiqueta, prediccion)

    # Obtener el gradiente de la pérdida respecto a la imagen
    gradiente = tape.gradient(loss, imagen)

    # Crear la perturbación usando el signo del gradiente
    perturbacion_adversaria = epsilon * tf.sign(gradiente)

    # Crear el ejemplo adversario
    imagen_adversaria = imagen + perturbacion_adversaria

    # Mantener los valores en el rango [0, 1]
    imagen_adversaria = tf.clip_by_value(imagen_adversaria, 0, 1)

    # Asegurar que devolvemos la misma forma que recibimos
    return imagen_adversaria.numpy().reshape(forma_original)

def visualizar_ejemplos(modelo, indice, epsilon):
    imagen_original = x_test[indice]
    etiqueta = y_test[indice]

    # Asegurarnos que la imagen original tiene la forma correcta para visualización
    imagen_original_vis = imagen_original.reshape(28, 28)

    # Crear imagen adversaria
    imagen_adversaria = crear_ejemplo_adversario_fgsm(modelo, imagen_original, etiqueta, epsilon)

    # Ajustamos la forma de la imagen adversaria para visualización
    # Esto es lo crítico, necesitamos asegurarnos de tener el tamaño correcto
    if imagen_adversaria.size != 784:  # 28*28
        print(f"La imagen adversaria tiene forma incorrecta: {imagen_adversaria.shape}, tamaño: {imagen_adversaria.size}")
        # Recortar o redimensionar si es necesario
        imagen_adversaria = imagen_adversaria.flatten()[:784].reshape(28, 28, 1)

    imagen_adversaria_vis = imagen_adversaria.reshape(28, 28)

    # Predicciones
    pred_original = modelo.predict(imagen_original.reshape(1, 28, 28, 1))
    pred_adversaria = modelo.predict(imagen_adversaria_vis.reshape(1, 28, 28, 1))

    pred_original_class = np.argmax(pred_original)
    pred_adversaria_class = np.argmax(pred_adversaria)

    # Visualización
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(imagen_original_vis, cmap='gray')
    plt.title(f"Original: {etiqueta}\nPredicción: {pred_original_class}")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(imagen_adversaria_vis, cmap='gray')
    plt.title(f"Adversario: {etiqueta}\nPredicción: {pred_adversaria_class}")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    # Calcular la perturbación (asegurando dimensiones iguales)
    perturbacion = imagen_adversaria_vis - imagen_original_vis
    plt.imshow(perturbacion, cmap='RdBu')
    plt.title(f"Perturbación (ε={epsilon})")
    plt.colorbar()
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Probar con diferentes ejemplos y epsilons
for idx in [10, 42, 56]:
    for eps in [0.05, 0.1, 0.2]:
        visualizar_ejemplos(model, idx, eps)

# Evaluar la robustez del modelo bajo ataque
def evaluar_bajo_ataque(modelo, epsilon=0.1, num_ejemplos=1000):
    # Seleccionar un subconjunto de ejemplos
    indices = np.random.choice(len(x_test), num_ejemplos, replace=False)
    x_subset = x_test[indices]
    y_subset = y_test[indices]

    # Crear ejemplos adversarios
    x_adv = np.zeros_like(x_subset)
    for i in range(len(x_subset)):
        x_adv[i] = crear_ejemplo_adversario_fgsm(modelo, x_subset[i], y_subset[i], epsilon)

    # Evaluar el modelo
    _, acc_original = modelo.evaluate(x_subset, y_subset, verbose=0)
    _, acc_adversario = modelo.evaluate(x_adv, y_subset, verbose=0)

    print(f"Epsilon: {epsilon}")
    print(f"Precisión en ejemplos originales: {acc_original:.4f}")
    print(f"Precisión en ejemplos adversarios: {acc_adversario:.4f}")
    print(f"Diferencia: {acc_original - acc_adversario:.4f}")

# Evaluar el modelo con diferentes intensidades de ataque
for eps in [0.01, 0.05, 0.1, 0.2, 0.3]:
    evaluar_bajo_ataque(model, epsilon=eps)